In [1]:
from utils import prepare_jupyter
prepare_jupyter()

In [24]:
import numpy as np
import networkx as nx

from IPython.display import Image
from structure import RandomForestEnsemble, Dataset

train_set, val_set = Dataset.create_iris().split(0.2)

params = {
    'n_estimators': 100,
    'max_depth': 2
}

rf = RandomForestEnsemble(params)
rf.fit(train_set)

tree_a, tree_b = rf.trees[2], rf.trees[4]

In [ ]:
# from itertools import product

# # nx_a = tree_a.tree
# # nx_b = tree_b.tree

# def compare_nodes(node_a, node_b):
#     if node_a['is_leaf'] and node_b['is_leaf']:
#         return node_a['target'] == node_b['target']
#     elif not node_a['is_leaf'] and not node_b['is_leaf']:
#         return node_a['threshold'] == node_b['threshold'] and node_a['feature'] == node_b['feature']
    
#     return False

# results = []
# for tree_a, tree_b in product(rf.trees, rf.trees):
#     if tree_a == tree_b:
#         continue
        
#     result = nx.graph_edit_distance(tree_a.tree, tree_b.tree, node_match=compare_nodes)
#     results.append(results)
    
# results = np.array(results)
# # nx.graph_edit_distance(nx_a, nx_b, node_match=compare_nodes)

In [3]:
tree_a.used_attrs, tree_b.used_attrs

({'0__5.950000047683716__$<=',
  '2__5.400000095367432__$<=',
  '3__0.800000011920929__$<=',
  '3__1.75__$<='},
 {'2__2.350000023841858__$<=', '3__1.550000011920929__$<=', '3__1.75__$<='})

In [4]:
rf.used_attr_diversity(), rf.used_feature_diversity()

AttributeError: 'RandomForestEnsemble' object has no attribute 'used_attr_diversity'

In [37]:
def get_mcs(G_source, G_new):
    matching_graph = nx.DiGraph()

    for n1, n2, _ in G_new.edges(data=True):
        if G_source.has_edge(n1, n2):
            matching_graph.add_edge(n1, n2, weight=1)

    graphs = list(nx.strongly_connected_component_subgraphs(matching_graph))

    mcs_length = 0
    mcs_graph = nx.DiGraph()
    
    for graph in graphs:
        if len(graph.nodes()) > mcs_length:
            mcs_length = len(graph.nodes())
            mcs_graph = graph

    return mcs_graph

In [38]:
new_g = get_mcs(tree_a.tree, tree_b.tree)
new_g

In [39]:
tree_a.draw('here.png')

In [40]:
tree_b.draw('here_b.png')

In [41]:
new_g.nodes(), new_g.edges()

(NodeView(('Leaf_1',)), OutEdgeView([]))

In [43]:
results = []

for i in range(len(rf)):
    tree_i = rf.trees[i].tree
    for j in range(i, len(rf)):
        tree_j = rf.trees[j].tree
    
        results.append(get_mcs(tree_i, tree_j))
        
max([len(x.nodes()) for x in results])


1